Import all required variables/ libraries from main

In [1]:
from main import data, pd, plt, train_test_split, GaussianNB, accuracy_score, DecisionTreeClassifier,  plot_tree, sns, apriori, association_rules, X_data, target, Dense, Sequential, export_graphviz, MinMaxScaler, RobustScaler, tree, gv
import constants
import utils
import pprint

pp = pprint.PrettyPrinter(indent=4)

2023-01-05 11:38:32.139859: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-05 11:38:32.256340: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-05 11:38:32.256361: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-05 11:38:32.938505: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Print data

In [ ]:
data

Print info about data (if null, occurrence, datatypes etc.)

In [ ]:
data.info()

Plots data, shows the distribution of MonkeyPox (how often it is negative or positive)

In [ ]:
series = data[constants.MONKEY_POX].value_counts()

plt.bar(series.index, series.values, color=constants.COLOR_PALETTE)
plt.title('Cases of Monkey Pox seperated by positive and negative')
plt.ylabel('Number of infections')
plt.show()

Shows True/ False distribution of all symptoms, containing bool-values

In [ ]:
symptoms = data.iloc[:, 1:-1].columns.values.tolist()
# Gets columns from dataframe and saves them as a list containing mulitple series
series_symptom_lst = [data[symptom].value_counts() for symptom in symptoms]
df_symptoms_series = pd.DataFrame()
# Generate dataframe from list of series and transposes it
df_symptoms_series= df_symptoms_series.append(series_symptom_lst).T
#df_symptoms_series = pd.DataFrame.from_records(series_symptom_lst).T
df_symptoms_series

Plots True/ False distribution of all symptoms, containing bool-values

In [ ]:
pos_lst, negative_lst = utils.create_outcome_lists(symptoms, df_symptoms_series, False, True)
# Plots pos_lst and negative_lst
utils.plot_data(symptoms, pos_lst, negative_lst, 'Symptoms', 'Number of infections', 'outcome', 'True', 'False')

Plots data, when features, containing bool-values are True and MonkeyPox is Positive

In [ ]:
symptoms_df = data.iloc[:, 1:]
symptoms_df = utils.fetch_true_symptoms(symptoms, symptoms_df)

pos_lst, negative_lst = utils.create_outcome_lists(symptoms, symptoms_df, constants.NEGATIVE, constants.POSITIVE)
utils.plot_data(symptoms, pos_lst, negative_lst, 'Symptoms', 'Number of infections', 'outcome', constants.POSITIVE, constants.NEGATIVE )

Plots distribution of column Systemic_Illness, using a bar chart

In [ ]:
sns.countplot(x=data[constants.SYSTEMIC_ILLNESS], hue=data[constants.MONKEY_POX], palette=constants.COLOR_PALETTE)
plt.show()

Shows correlations between the different features

In [ ]:
df_corr  = data.corr(numeric_only=True)
df_corr

Plots correlation-dataframe, by using a heatmap

In [ ]:
sns.heatmap(df_corr, annot=True, vmin=0.5, vmax=0.7)
plt.show()

Alters dataframe, values of the feature: Systemic_Illness are added as new columns

In [ ]:
# Dataframe, containing the values from feature: Systemic_Illness as columns
symptoms_df = utils.clean_data(data)
symptoms_df

Calculates lift, confidence, support etc. for all possible feature combinations

In [ ]:
frequency = apriori(symptoms_df, min_support=0.005, use_colnames=True)
rules = association_rules(frequency, metric='lift')
rules.sort_values('confidence', ascending=False, inplace=True)
rules

Calculates accuracy, using GaussianNB()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, target)

model = GaussianNB()
model.fit(X_train, y_train)
y_prediction = model.predict(X_test)
print('Accuracy: {:.2f}'.format(100 * accuracy_score(y_test, y_prediction)))
print('R² of train-data: {:.2f}'.format(model.score(X_train, y_train)))
print('R² of test-data: {:.2f}'.format(model.score(X_test, y_test)))

Creates and plots DecisionTree, by using entropy

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, target, test_size=0.30)
model_tree = DecisionTreeClassifier(criterion='entropy', splitter='best', min_samples_split=0.20)
model_tree.fit(X_train, y_train)
#
# y_prediction = model_tree.predict(X_test)

# Plot decisiontree
# figure = plt.figure(figsize=(10,8))
# plot_tree(model_tree, feature_names=X_train.columns, filled=True, rounded=True)
# plt.show()
tree.dot = export_graphviz(model_tree, filled=True, feature_names=X_test.columns)
graph = gv.Source(tree.dot)
graph

Calculates accuracy for all possible feature combinations, using three different models

In [ ]:
acc_dict = utils.create_feature_accuracy_dict(X_data, target)
# Sort dictionary in a descending manner and save the top 5 elements
top_items = sorted(acc_dict.items(), key = lambda x: x[1], reverse=True)[:5]
pp.pprint(top_items)

Tries to detect overfitting by plotting the test and train data

In [ ]:
utils.check_over_fitting(X_data, target)

Builds neural network, which contains three hidden layers.

In [40]:
model = Sequential()

X, y = utils.prepare_for_tensor(X_data, target)

# Input-layer 9 features = 9 neurons
model.add(Dense(9, input_dim=X_data.shape[1], activation='sigmoid', name='input'))

# 35 hidden nodes in the first and second layer
model.add(Dense(15, activation='relu', name='layer1'))
model.add(Dense(15, activation='relu', name='layer2'))
model.add(Dense(15, activation='relu', name='layer3'))
model.add(Dense(1, activation='sigmoid', name='output'))

model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y)


586/586 [==============================] - 1s 820us/step - loss: 0.2434 - accuracy: 0.6370


Print summary of the NN model

In [41]:
model.summary()

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Dense)               (None, 9)                 90        
                                                                 
 layer1 (Dense)              (None, 15)                150       
                                                                 
 layer2 (Dense)              (None, 15)                240       
                                                                 
 layer3 (Dense)              (None, 15)                240       
                                                                 
 output (Dense)              (None, 1)                 16        
                                                                 
Total params: 736
Trainable params: 736
Non-trainable params: 0
_________________________________________________________________


Evaluates quality of train- and test-data from above

In [44]:
hist = model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0, validation_data=(X_test, y_test))
print('Train-data:')
# Evaluate train-data
utils.eval_data(X_train, y_train, model)
print('Test-data:')
# Evaluate test-data
utils.eval_data(X_test, y_test, model)

Train-data:
586/586 [==============================] - 1s 860us/step
loss: 0.20
accuracy: 0.70
Test-data:
196/196 [==============================] - 0s 828us/step
loss: 0.20
accuracy: 0.69


Plots the neural network

In [45]:
evaluation = pd.DataFrame.from_dict(hist.history)
fig = plt.figure(figsize=(20, 8), num='Neural Network')
img1 = fig.add_subplot(121)
img1.plot(evaluation.index, evaluation.iloc[:, 0], color=constants.FIRST_COLOR, label='Train-data')
img1.plot(evaluation.index, evaluation.iloc[:, 2], color=constants.SECOND_COLOR, label='Test-data')
img1.legend()
img1.set_xlabel('epoch')
img1.set_ylabel(model.metrics_names[0])
img1.set_title('Neural Network learns: Loss curve')
img2 = fig.add_subplot(122)
img2.plot(evaluation.index, evaluation.iloc[:, 1], color=constants.FIRST_COLOR, label='Train-data')
img2.plot(evaluation.index, evaluation.iloc[:, 3], color=constants.SECOND_COLOR, label='Test-data')
img2.legend()
img2.set_xlabel('epoch')
img2.set_ylabel(model.metrics_names[1])
img2.set_title('Neural Network learns: Accuracy curve')
plt.show()